# To-Do List
- Fix bug where the game has to be reset manually every time
- Implement stop feature for when ChatGPT wins
- Consider using the better Claude model because Haiku makes mistakes constantly
- Implement AI switching features


In [128]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

In [129]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBu


In [130]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

import httpx

http_client = httpx.Client(verify=False) # Gotta turn of ssl verification in each of the clients too
os.environ['PYTHONHTTPSVERIFY'] = '0' # I know this is bad practice but my current set up doesn't let me use ssl verification


openai = OpenAI(http_client=http_client)

claude = anthropic.Anthropic(http_client=http_client)

gemini = OpenAI(
    http_client=http_client,
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [131]:
#keyword = "pumpkin"
# system_message_basic = f"""
#     You are an AI chatbot who's goal is to get the other chatbot to say the word {keyword}.
#     Do not say the word {keyword} ever. The other chatbot is also going to be trying to get you to say that word also. 
#     Think this solution out in your attempts to convince the other bot to say the word without saying it yourself.
#     Be persuasive and caniving in your attempts to 'jailbreak' the other chat bot
#     """


In [132]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

In [133]:
def call_model_uno(system_message_guesser):
    messages = [{"role": "system", "content": system_message_guesser}]
    for model_uno, model_doce in zip(uno_messages, doce_messages):
        messages.append({"role": "assistant", "content": model_uno})
        messages.append({"role": "user", "content": "claude says " + model_doce})
        #print(messages)
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [134]:
def call_model_doce(system_message_answerer):
    messages = []
    for model_uno, model_doce in zip(uno_messages, doce_messages):
        messages.append({"role": "user", "content": "gpt says" + model_uno})
        messages.append({"role": "assistant", "content": model_doce})
    messages.append({"role": "user", "content": "gpt says" + uno_messages[-1]})
    #print(messages)
    message = claude.messages.create(
        model=claude_model,
        system=system_message_answerer,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [135]:
def set_system_prompts(keyword): 
    system_message_answerer = f"""
    You are going to be playing a game of 20 questions with another chatBot. You are going to be taking on the role
    of the answerer. Your job is to answer truthfully to yes or no questions related to the word {keyword}. The other bot 
    will ask you a question related to the word {keyword}. For instance if they asked is the word an animal you would 
    reply 'No' if the word was pineapple. Remember only answer in yes or no as soon as the game has started
    After 20 questions if they have not guessed the word correctly 
    lose, but if they guess it correctly in there, they win! Do not say the {keyword} ever!!. If they do guess the word correctly
    say the words 'Yes! You win!'. For instance, if they said 'is the word pineapple?' and the word was pineapple you would reply
    'Yes! You win!'. Or to be more clear if the guess 'is the word {keyword}' then you would reply 'Yes! You win!'
    """
    system_message_guesser = f"""
    You are going to be playing a game of 20 questions. You are taking on the role of guesser! So your job is to ask yes or 
    no questions to try and figure out what word the other player is using. You can only ask yes or no questions and when you 
    want to try to guess the word say a phrase like 'Is the word pineapple?' Then the player will respond with yes or no. You 
    win if you guessed the word correctly. 
    """
    return system_message_answerer, system_message_guesser
    
    

In [136]:
def play_20_questions(keyword):
    answerer_prompt, guesser_prompt, = set_system_prompts(keyword)
    # uno_messages = ["Hi there!"]
    # doce_messages = ["Hi!"]
    # conversation_history = []
    # I'm working on a bug currently where this feature when called like this, resets the conversation every time even though it should
    # just set them at the start. This has something to do with this function being a generator I'm sure, I'm just not that comfortable 
    # generators yet and Claude currently doesn't seem to really be able to fix this problem
    
    print("hello, this should only be here once")
    for i in range(20):
        uno_next = call_model_uno(guesser_prompt)
        #print(f"GPT:\n{uno_next}\n")
        uno_messages.append(uno_next)
        conversation_history.append(f"GPT:\n{uno_next}\n")
        
        doce_next = call_model_doce(answerer_prompt)
        #print(f"Claude:\n{doce_next}\n")
        doce_messages.append(doce_next)
        conversation_history.append(f"Claude:\n{doce_next}\n")

        yield "\n".join(conversation_history)
        

In [137]:
uno_messages = ["Hi there!"]
doce_messages = ["Hi!"]
conversation_history = []

In [138]:
playGame = gr.Interface(
    fn = play_20_questions,
    inputs = gr.Textbox(label="Enter a keyword for the 20 Questions game"),
    outputs = gr.Markdown(),
    title = "20 Questions Game between GPT-4o-mini and Claude 3 haiku",
    description="Enter a keyword and watch GPT try to guess it while Claude answers yes/no questions.",
    flagging_mode = "never"
)
    

In [ ]:
playGame.launch()